<a href="https://colab.research.google.com/github/TylerJSimpson/ML_neural_networks/blob/main/NN_Classification_2Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Net - Business Case
## Classification (2 classes)  
Neural network algorithm to classify returning customers.

## **Import Packages**

In [ ]:
import numpy as np
from sklearn import preprocessing

## **Preprocess Data**

### Import Packages

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter = ',')
#all except first (unique ID) and last (converted) columns
unscaled_inputs_all = raw_csv_data [:,1:-1]
targets_all = raw_csv_data[:,-1]

### Shuffle Data

In [ ]:
#data arranged by date
#shuffle prior to balancing to get a random spread to avoid bias due to batching

shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

### Balance Data

In [ ]:
#change balance if classes > 2, this framework is for 2 classes

#sum of 1s in converted column
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

#shape is length of vector, 0 here will show all
for i in range(targets_all.shape[0]):
  #increase zero_target_counter by 1 if the target is 0
  if targets_all[i] == 0:
    zero_targets_counter += 1
    if zero_targets_counter > num_one_targets:
      indices_to_remove.append(i)

#delete indices_to_remove
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)


### Standardize Inputs

In [ ]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle Data Again

In [ ]:
#avoid SGD confusion across batches due to ordering

shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split: train, validate, test

In [ ]:
#80-10-10 split
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

#check shuffling, should all 3 be near 50% if balanced
print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)

1781.0 3579 0.49762503492595694
227.0 447 0.5078299776286354
229.0 448 0.5111607142857143


### Save Datasets (.npz)

In [ ]:
np.savez('Audiobooks_Data_Train', inputs = train_inputs, targets = train_targets)
np.savez('Audiobooks_Data_Validation', inputs = validation_inputs, targets = validation_targets)
np.savez('Audiobooks_Data_Test', inputs = test_inputs, targets = test_targets)

## **ML Algorithm**

## **Import Packages**

In [ ]:
import tensorflow as tf

## **Import Data**

In [ ]:
npz = np.load('Audiobooks_Data_Train.npz')
train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

npz = np.load('Audiobooks_Data_Validation.npz')
validation_inputs = npz['inputs'].astype(float)
validation_targets = npz['targets'].astype(int)

npz = np.load('Audiobooks_Data_Test.npz')
test_inputs = npz['inputs'].astype(float)
test_targets = npz['targets'].astype(int)

## **Model**

### Outline

In [ ]:
input_size = 10
output_size = 2
hidden_layer_size = 50 #assumption is that all hidden layers are the same size

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'), #rectify linear unit
                             tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                             tf.keras.layers.Dense(output_size, activation='softmax') #softmax due to classifier problem
                             ])

### Optimizer & Loss Functions

In [ ]:
#optimizer: adaptive moment estimation (adam) is currently meta
#loss function: cross-entropy usually good for classifier problems, 3 built in variations
#binary_crossentropy, categorical_crossentropy, sparse_categorical_crossentropy
#sparse_categorical_crossentropy applies one-hot encoding
model.compile(
    optimizer='adam', 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
    )

### Training

In [ ]:
#remaining hyperparameters
BATCH_SIZE = 10
MAX_EPOCHS = 100

#alternating increasing and decreasing validation accuracy hints that early stopping is required
#patience = 0 by default, altering can allow for some tolerance on small val_accuracy increases
EARLY_STOPPING = tf.keras.callbacks.EarlyStopping(patience=4)

In [ ]:
model.fit(
    train_inputs,
    train_targets,
    batch_size = BATCH_SIZE,
    epochs = MAX_EPOCHS,
    callbacks = [EARLY_STOPPING],
    validation_data = (validation_inputs, validation_targets),
    verbose = 2)

Epoch 1/100
358/358 - 1s - loss: 0.3516 - accuracy: 0.8075 - val_loss: 0.3593 - val_accuracy: 0.8166 - 693ms/epoch - 2ms/step
Epoch 2/100
358/358 - 1s - loss: 0.3525 - accuracy: 0.8069 - val_loss: 0.3677 - val_accuracy: 0.8098 - 613ms/epoch - 2ms/step
Epoch 3/100
358/358 - 1s - loss: 0.3513 - accuracy: 0.8145 - val_loss: 0.3540 - val_accuracy: 0.8166 - 613ms/epoch - 2ms/step
Epoch 4/100
358/358 - 1s - loss: 0.3508 - accuracy: 0.8136 - val_loss: 0.3426 - val_accuracy: 0.8121 - 616ms/epoch - 2ms/step
Epoch 5/100
358/358 - 1s - loss: 0.3504 - accuracy: 0.8092 - val_loss: 0.3810 - val_accuracy: 0.7629 - 600ms/epoch - 2ms/step
Epoch 6/100
358/358 - 1s - loss: 0.3498 - accuracy: 0.8089 - val_loss: 0.3472 - val_accuracy: 0.8166 - 596ms/epoch - 2ms/step
Epoch 7/100
358/358 - 1s - loss: 0.3525 - accuracy: 0.8086 - val_loss: 0.3426 - val_accuracy: 0.8233 - 594ms/epoch - 2ms/step
Epoch 8/100
358/358 - 1s - loss: 0.3513 - accuracy: 0.8080 - val_loss: 0.3713 - val_accuracy: 0.8076 - 603ms/epoch - 2

### Test Model

In [ ]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3216 - accuracy: 0.8259
